In [91]:
import pennylane as qml
from scipy.optimize import minimize
from pennylane import numpy as np




In [92]:
n_qubits = 8
dev = qml.device('default.qubit', wires=n_qubits)

In [93]:
def find_n_largest_numbers(arr, n):
    # Find the indices of the n largest elements
    indices = np.argsort(arr)[-n:]
    # Find the n largest elements
    largest_elements = arr[indices]
    return largest_elements, indices

def indices_to_binary(indices):
    # Convert each index to binary and ensure it's 8 bits
    return [format(index, f'0{n_qubits}b') for index in indices]

def int_to_binary(n, digits):
    binary_str = format(n, 'b')
    # Pad the binary string with leading zeros to ensure it has 'digits' length
    return binary_str.zfill(digits)

In [94]:
# Define the quantum circuit
@qml.qnode(dev)
def circuit(param,index):
    index = int_to_binary(index,n_qubits)
    for i, char in enumerate(index):
        if(char=='1'):
            qml.PauliX(wires=i)
            
    qml.StronglyEntanglingLayers(weights=param, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

In [95]:
a = int_to_binary(4,5)
for i, char in enumerate(a):
    if(char=='1'):
        print(True)

True


In [96]:
# Define the shape of the parameters
shape_p = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=n_qubits)

# Define the objective function (to be minimized)
def objective(params):
    print(params)
    original_length = np.prod(shape_p)
    # Extract segments for each set of weights
    weights_p1_flat = params[0 : original_length]
    # weights_p2_flat = params[original_length : 2*original_length]
    # weights_p3_flat = params[2*original_length : 3*original_length]
    # weights_p4_flat = params[3*original_length : 4*original_length]

    # Reshape each segment
    weights_p1 = weights_p1_flat.reshape(shape_p)
    # print(weights_p1)
    # weights_p2 = weights_p2_flat.reshape(shape_p)
    # weights_p3 = weights_p3_flat.reshape(shape_p)
    # weights_p4 = weights_p4_flat.reshape(shape_p)
    print('===================')
    arr = circuit(weights_p1,1)
    largest_elements, indices = find_n_largest_numbers(arr, 1)
    # Convert indices to binary
    binary_indices = indices_to_binary(indices)
    print(binary_indices[0])

    image_arr = []
    # for i in range(n_qubits):

    print(arr)
    # Find n largest numbers and their indices
    



    # print("Largest elements:", largest_elements)
    # print("Indices:", indices)
    # print("Indices in binary:", binary_indices)

    return circuit(weights_p1)

def callback(params):
    print("Current loss:", objective(params))
    
# Initialize the weights
weights_p1 = np.random.random(size=shape_p)
# weights_p2 = np.random.random(size=shape_p)
# weights_p3 = np.random.random(size=shape_p)
# weights_p4 = np.random.random(size=shape_p)

# Flatten and concatenate the initial parameters
init_params = np.concatenate([weights_p1.flatten()])


# Optimize using BFGS algorithm
opt = minimize(objective, init_params, method='BFGS',callback=callback)


[0.19153571 0.19628261 0.6438483  0.76003188 0.12074209 0.09557529
 0.52376949 0.26483429 0.0650138  0.54072174 0.20011191 0.13896518
 0.0822253  0.21179058 0.53875331 0.42306429 0.58700861 0.33053178
 0.51453728 0.76150167 0.0883673  0.93733918 0.00885047 0.00805532
 0.23073136 0.24039391 0.81417845 0.7148313  0.04083461 0.16099664
 0.97635008 0.26470443 0.19097625 0.71236344 0.31527361 0.4010627
 0.00839654 0.30245661 0.24956138 0.96735738 0.50402995 0.34009899
 0.92544593 0.40565915 0.24927774 0.55019772 0.7015923  0.58313125]
01101011
[8.29789139e-04 2.85059086e-06 7.95253802e-06 4.83129024e-06
 1.72237865e-03 6.93769542e-08 5.55578635e-05 8.64430812e-05
 2.44715071e-03 2.14220170e-07 4.68080392e-05 1.73996533e-04
 1.45095224e-03 2.88839395e-05 1.27126812e-04 5.61746813e-04
 3.19214253e-05 4.04973471e-06 1.09193031e-06 2.24654629e-06
 2.91241522e-04 1.45279893e-05 7.03177195e-06 9.13296505e-06
 7.62361678e-03 1.17695134e-06 4.27013097e-04 1.07594175e-05
 3.64633729e-05 1.86957117e-

TypeError: circuit() missing 1 required positional argument: 'index'

In [ ]:
print("Optimized Parameters:", opt.x)
print("Optimized Expectation Value:", opt.fun)

original_length = np.prod(shape_p)
params = opt.x
# Extract segments for each set of weights
weights_p1_flat = params[0 : original_length]
# weights_p2_flat = params[original_length : 2*original_length]
# weights_p3_flat = params[2*original_length : 3*original_length]
# weights_p4_flat = params[3*original_length : 4*original_length]

# Reshape each segment
weights_p1 = weights_p1_flat.reshape(shape_p)
# weights_p2 = weights_p2_flat.reshape(shape_p)
# weights_p3 = weights_p3_flat.reshape(shape_p)
# weights_p4 = weights_p4_flat.reshape(shape_p)
# print(circuit_res(weights_p1,weights_p2,weights_p3,weights_p4)

In [ ]:
res = circuit(p1=weights_p1)
print(res)

In [ ]:

# Example array
arr = res
n = 3

# Find n largest numbers and their indices
largest_elements, indices = find_n_largest_numbers(arr, n)

# Convert indices to binary
binary_indices = indices_to_binary(indices)

print("Largest elements:", largest_elements)
print("Indices:", indices)
print("Indices in binary:", binary_indices)
